To install libraries

In [0]:
pip install b4

In [0]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

Code 

In [0]:
import requests 
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords

query = input("Enter word to search: ")
url = f"https://google.com/search?q={query}" 
res = requests.get(url, headers={"User-Agent":"Mozilla/0.5"})

soup = BeautifulSoup(res.content, "html.parser")
result_div = soup.find_all('div', attrs = {'class': 'ZINbbc'})

links = []
titles = []
descriptions = []

for r in result_div:
    try:
        link = r.find('a', href = True)
        title = r.find('div', attrs={'class':'vvjwJb'}).get_text()
        description = r.find('div', attrs={'class':'s3v9rd'}).get_text()

        if link != '' and title != '' and description != '': 
            links.append(link['href'])
            titles.append(title)
            descriptions.append(description)
    except:
        continue
        
to_remove = []
clean_links = []
for i, l in enumerate(links):
    clean = re.search('\/url\?q\=(.*)\&sa',l)
    
    if clean is None:
        to_remove.append(i)
        continue
    clean_links.append(clean.group(1))

for x in to_remove:
    del titles[x]
    del descriptions[x]
    
    
List_all_rank = []
def Sort_Tuple(tup,ind):
    tup.sort(key = lambda x: x[ind])  
    return tup  

def lesk(query, sentence,ind):
    Text1 = sentence.lower()
    words = nltk.word_tokenize(Text1)
    
    stop_words = stopwords.words("english")
    stop_words += ['can', 'will', 'use', 'one', 'using', 'used', 'also', 'see', 'first', 'like']
    stop_words += ['page', 'get', 'new', 'two', 'site', 'blog', 'many', 'may' ,"don't", 'dont', 'way']
    stop_words += ['last', 'best', 'able', 'even', 'next', 'last', 'let', "none", 'every', 'three']
    stop_words += ['lot', 'well', 'chart', 'much', 'based', 'important', 'posts', 'reads', 'least']
    stop_words += ['still', 'follow', 'called', 'and','this', 'that', 'there', 'as','the', 'is']
    stop_words += ['/', '=', '.', ',', '.']

    filtered_words = []
    for word in words:
        if word not in stop_words:
            filtered_words.append(word)
    word = query
    len_syn = len(wordnet.synsets(word))
    
    da=[]
    ea=[]
    List_rank = []
    for x in range(len_syn):

        for i in filtered_words:
            synset = wordnet.synsets(word)[x]
            if i in synset.definition().lower():
                if i in da:
                    pass
                else:
                    da.append(i)
              
            if len(synset.examples()) == 0:
                pass
            else:
                if i in synset.examples()[0].lower() and word in synset.examples()[0].lower():
                    if i in ea:
                        pass
                    else:
                        ea.append(i)
                        
        l = []
        l.insert(0,x)
        r = len(da)+len(ea)
        l.append(r)
        y2 = tuple(l)
        List_rank.append(y2)
    Sorted_list_rank = Sort_Tuple(List_rank,1)
    l = list(Sorted_list_rank[-1])
    l.insert(0,ind)
    y2 = tuple(l)                           
    List_all_rank.append(y2)
    
for idx, val in enumerate(descriptions):
    lesk(query, val, idx)

s_li = Sort_Tuple(List_all_rank,2)
for x in range(len(List_all_rank),0,-1):
    print("Page Number",s_li[x-1][0]+1, ", Most Synset Num",s_li[x-1][1],", With Rank",s_li[x-1][2])
    print("Title:",titles[s_li[x-1][0]])
    print("Link:",clean_links[s_li[x-1][0]])
    print("-------------------------\n")